In [213]:
import pandas as pd
import altair as alt
import warnings
import requests
import datetime
import matplotlib.pyplot as plt
import json
import requests
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
alt.renderers.set_embed_options(theme='dark')
pd.set_option('display.max_colwidth', None)

In [214]:
N_PROPOSALS = 8

In [231]:
class AstroDataProvider:
    
    def __init__(self, claim):
        daic_url = "https://terra-api.daic.capital/api/tx/GetRichlistByTokenContract?apiKey=vAp6ysmAXH470YcphYxv&contract_address={}"
        self.votes = '4940a215-6e93-4107-bf08-50574b3e431d'
        self.astro_holders_url =daic_url.format("terra1xj49zyqrwpv5k928jwfpfy2ha668nwdgkwlrg3")
        self.xastro_holders_url =daic_url.format("terra14lpnyzc9z4g3ugr4lhm8s4nle0tq8vcltkhzh7")
        self.claim = claim
        
    def get_from_url(self, url):
        json = requests.get(url).json()
        return json
        
    def load(self):
        self.votes_df = self.claim(self.votes)
        #
        json = self.get_from_url(self.astro_holders_url)['result']['holders']
        self.astro_holders_df = pd.DataFrame(json.values(),json.keys()).reset_index()
        self.astro_holders_df.columns =  ['addr','amount']
        #
        json = self.get_from_url(self.xastro_holders_url)['result']['holders']
        self.xastro_holders_df = pd.DataFrame(json.values(),json.keys()).reset_index()
        self.xastro_holders_df.columns =  ['addr','amount']
        #
        
    def parse_proposal_recap(self):
        votes = self.votes_df.groupby(['proposal_id','vote']).sum().voting_power.reset_index()
        against = votes[votes.vote=='against']
        against.columns = ['proposal_id','against','voting_power_against']
        for_ = votes[votes.vote=='for']
        for_.columns = ['proposal_id','for','voting_power_for']
        votes = against.merge(for_, on='proposal_id')
        votes['delta'] = votes['voting_power_for'] - votes['voting_power_against'] 
        votes['result'] = votes.apply(lambda row: 'passed' if row.delta > 0 else 'failed', axis=1)
        votes['result'] = votes.apply(lambda row: 'passed' if row.delta > 0 else 'failed', axis=1)
        return votes
    
    def parse_top_active_voters(self):
        return dp.votes_df.groupby('voter').agg({'voting_power':'sum','tx_id':'count'})\
                            .sort_values(by=['tx_id','voting_power'], ascending=False)\
                            .head(20)
    def parse_dist_voting_power_per_proposal(self):
        return dp.votes_df[['proposal_id','voting_power']]\
            .pivot(columns='proposal_id',values='voting_power')
    
    def parse_top_voters_per_proposal(self):
        df=[]
        for i in range(1,N_PROPOSALS):
            if(len(df)==0):
                df = dp.votes_df[dp.votes_df.proposal_id==1].sort_values(by='voting_power', ascending=False).head(10)
            else:
                df = df.append(dp.votes_df[dp.votes_df.proposal_id==i].sort_values(by='voting_power', ascending=False).head(10))
        return df
    
    def parse_votes_over_time(self):
        df_ = []
        for i in range(1,N_PROPOSALS):
            df = self.votes_df[dp.votes_df.proposal_id==i].groupby(['hr','proposal_id','vote']).voting_power.sum().reset_index()
            df_for = df[df.vote=='for']
            df_for.columns = ['hr','proposal_id','vote_for','voting_power_for']
            df_against = df[df.vote=='against']
            df_against.columns = ['hr','proposal_id','vote_against','voting_power_against']
            df = df_for.merge(df_against, on=['hr','proposal_id'], how='outer')
            df.vote_against = df.vote_against.fillna('against')
            df.vote_for = df.vote_for.fillna('for')
            df.voting_power_against = df.voting_power_against.fillna(0)
            df.voting_power_for = df.voting_power_for.fillna(0)
            df['voting_power_for_cumsum'] = df.sort_values(by=['hr']).voting_power_for.cumsum()
            df['voting_power_against_cumsum'] = df.sort_values(by=['hr']).voting_power_against.cumsum()
            if(len(df_)==0):
                df_ = df
            else:
                df_ = df_.append(df)
        return df_
    
    def parse_voting_power_cumulative(self):
        df_ = []
        top_various_prop = []
        for i in range(1,N_PROPOSALS):
            df = dp.votes_df
            df = df[df.proposal_id==i]
            df['voting_power_perc'] = df.voting_power/df.voting_power.sum()*100
            df = df.sort_values(by='voting_power_perc', ascending=False)
            df['voting_power_cumsum'] = df.voting_power_perc.fillna(0).cumsum()
            df['n_addresses'] = np.arange(len(df))+1
            assert (int(df.voting_power_cumsum.max()) in [100,99]), 'Total % should give 100'
            top_various_prop.append((i,len(df[df.voting_power_cumsum<50])+1))
            if(len(df_)==0):
                    df_ = df
            else:
                df_ = df_.append(df)
        top_various_prop_df = pd.DataFrame(top_various_prop)
        
        return df_top_various_prop_df
    
    def parse(self):
        df = self.votes_df
        df['block_timestamp'] = df.block_timestamp.astype('datetime64[ms]')
        df.block_timestamp=df.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        df.block_timestamp=df.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        df.block_timestamp=df.block_timestamp.apply(str).apply(lambda x: x[:-7] if len(x) == 26 else x)
        self.votes_df = df
        #
        dp.votes_df['hr'] = dp.votes_df.block_timestamp.apply(str).str[:-5] + '00'
        dp.votes_df['day'] = dp.votes_df.block_timestamp.apply(str).str[:-9]
        self.astro_holders_df.amount = self.astro_holders_df.amount/1000000
        self.proposal_recap = self.parse_proposal_recap()
        self.top_active_voters = self.parse_top_active_voters()
        self.dist_voting_power_per_proposal = self.parse_dist_voting_power_per_proposal()
        self.top_voters_per_proposal = self.parse_top_voters_per_proposal()
        self.votes_over_time = self.parse_votes_over_time()
        self.voting_power_cumulative = self.parse_voting_power_cumulative()
        
    def to_file(self, path='../data'):
        self.votes_df.to_json(f"{path}/votes_df",orient='records')
        self.astro_holders_df.to_json(f"{path}/astro_holders_df",orient='records')
        self.proposal_recap.to_json(f"{path}/proposal_recap", orient='records')
        self.top_active_voters.to_json(f"{path}/top_active_voters",orient='records')
        self.dist_voting_power_per_proposal.to_json(f"{path}/dist_voting_power_per_proposal",orient='records')
        self.top_voters_per_proposal.to_json(f"{path}/top_voters_per_proposal",orient='records')
        self.votes_over_time.to_json(f"{path}/votes_over_time",orient='records')
        self.voting_power_cumulative.to_json(f"{path}/voting_power_cumulative",orient='records')
        
    def read_file(self):
        url = 'https://raw.githubusercontent.com/IncioMan/astroport_governance/master/data/{}'
        self.votes_df =  pd.read_json(url.format('votes_df'))
        self.astro_holders_df =  pd.read_json(url.format('astro_holders_df'))
        self.proposal_recap =  pd.read_json(url.format('proposal_recap'))
        self.top_active_voters =  pd.read_json(url.format('top_active_voters'))
        self.dist_voting_power_per_proposal =  pd.read_json(url.format('dist_voting_power_per_proposal'))
        self.top_voters_per_proposal =  pd.read_json(url.format('top_voters_per_proposal'))
        self.votes_over_time =  pd.read_json(url.format('votes_over_time'))
        self.voting_power_cumulative =  pd.read_json(url.format('voting_power_cumulative'))


In [232]:
def claim(claim_hash):
    df = pd.read_json(
            f"https://api.flipsidecrypto.com/api/v2/queries/{claim_hash}/data/latest",
            convert_dates=["BLOCK_TIMESTAMP"])
    df.columns = [c.lower() for c in df.columns]
    return df

In [233]:
dp = AstroDataProvider(claim)
dp.load()
dp.parse()
dp.to_file()
#dp.read_file()

/var/folders/pd/3vx1d3qj29z1l1tm70p573s80000gn/T/ipykernel_23303/3382957454.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['voting_power_perc'] = df.voting_power/df.voting_power.sum()*100
/var/folders/pd/3vx1d3qj29z1l1tm70p573s80000gn/T/ipykernel_23303/3382957454.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['voting_power_perc'] = df.voting_power/df.voting_power.sum()*100
/var/folders/pd/3vx1d3qj29z1l1tm70p573s80000gn/T/ipykernel_23303/3382957454.py:81: SettingWithCopyWarning: 
A value is

In [237]:
i=1
df = dp.votes_df
df = df[df.proposal_id==i]
df['voting_power_perc'] = df.voting_power/df.voting_power.sum()*100
df = df.sort_values(by='voting_power_perc', ascending=False)
df['voting_power_cumsum'] = df.voting_power_perc.fillna(0).cumsum()
df['n_addresses'] = np.arange(len(df))+1
assert (int(df.voting_power_cumsum.max()) in [100,99]), 'Total % should give 100'

/var/folders/pd/3vx1d3qj29z1l1tm70p573s80000gn/T/ipykernel_23303/3197584954.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['voting_power_perc'] = df.voting_power/df.voting_power.sum()*100


,0,1
0,1,3


In [235]:
dp = AstroDataProvider(claim)
#dp.load()
#dp.parse()
#dp.to_file()
dp.read_file()

In [234]:
dp.voting_power_cumulative[dp.voting_power_cumulative.proposal_id==3]

,block_timestamp,tx_id,action,proposal_id,vote,voter,voting_power,hr,day,voting_power_perc,voting_power_cumsum,n_addresses
7448,2022-04-22 14:46:11,DB0EB3EDD43773AC4AE11EC5046947065E347C9E1F95A3D0060418810EB02E0A,cast_vote,3,against,terra1zaqeperrwghqlsa9yykzsjaets54mtq0u6kl60,1.500000e+13,2022-04-22 14:00,2022-04-22,3.430098e+01,34.300982,1
5348,2022-04-21 11:54:43,39BE6B1475CC256A99F303879A44B19A85B7CACEA2E4005500DC7523C8B6C6E4,cast_vote,3,for,terra1pq02fnrm68x6kcv2lhgvyetjelps550w3pq6m2,5.000000e+12,2022-04-21 11:00,2022-04-21,1.143366e+01,45.734643,2
2468,2022-04-21 20:22:20,7D97996C1A387A58028B655BC7345D9053CA09EE815751EFFDFABE4DD52B55E8,cast_vote,3,against,terra1jyxljeq865wsndutyyr0vrzzq9tj3fa6rmhgrl,3.774557e+12,2022-04-21 20:00,2022-04-21,8.631400e+00,54.366043,3
2474,2022-04-21 20:12:36,32BEC3378FFA792C68D53DBA8B3582AB4226C6FC7FD429BB617500A23769B0B1,cast_vote,3,against,terra1qn93rzsnfh0dwdkxmf8q6dtjr0uxv33klhlc5u,3.403807e+12,2022-04-21 20:00,2022-04-21,7.783594e+00,62.149637,4
6677,2022-04-23 20:20:03,81C4690F4E0E16D7D154ADA21B7527925665ACB0D5B791D86D2C4CF812EEF536,cast_vote,3,against,terra1gn53cj0v8kvwxqg867e3mu9f3q9yzskmfgnvla,2.007961e+12,2022-04-23 20:00,2022-04-23,4.591670e+00,66.741307,5
...,...,...,...,...,...,...,...,...,...,...,...,...
5813,2022-04-21 04:23:25,C99F69BF511EBD66BC10FE77DCEF57617CE69681FE6ECBB2AAE3140052E65054,cast_vote,3,for,terra1c4z2qv69nlg9wk72aj9jeryay3uft0he3znxr2,3.952380e+05,2022-04-21 04:00,2022-04-21,9.038034e-07,99.999999,1181
6420,2022-04-21 13:43:04,95DBBA7E16CB40CBF07251558A8C2919049E1BF2EC1E107403DA95BF1BB394B3,cast_vote,3,for,terra1ptyq4ggfdpy8egsju0xwx0q9zthh5sy0pncvyv,3.072440e+05,2022-04-21 13:00,2022-04-21,7.025847e-07,100.000000,1182
7099,2022-04-23 19:16:11,F542AEA57C2B4E9C021B1A3841E3EFB241B46B60E966AAD9098FA36395C84C0B,cast_vote,3,against,terra1sm8dmak34ay2acly50hwqjrmh56rle5msk0j27,3.990600e+04,2022-04-23 19:00,2022-04-23,9.125433e-08,100.000000,1183
2353,2022-04-20 18:13:52,DCD5CD0A979B31A2BBD03782CE714C8E34E04A996D3DD000946F08896B8438E2,cast_vote,3,for,terra1l35xhpjnyqu86j6k59lqnw9mcavfr9vzhseyw7,2.711100e+04,2022-04-20 18:00,2022-04-20,6.199560e-08,100.000000,1184
